# 11장 인공지능 이미지 생성기

## 11.1 번역 텍스트로 이미지 생성

[11장: 298페이지]

In [ ]:
import openai
import os

# OpenAI Chat Completions API를 이용해 한국어를 영어로 번역하는 함수
def translate_text_for_image(text):    
    # API 키 설정
    openai.api_key = os.environ["OPENAI_API_KEY"]

    # 대화 메시지 정의
    user_content = f"Translate the following Korean sentences into English.\n {text}"
    messages = [ {"role": "user", "content": user_content} ]

    # Chat Completions API 호출
    response = openai.ChatCompletion.create(
                            model="gpt-3.5-turbo", # 사용할 모델 선택 
                            messages=messages, # 전달할 메시지 지정
                            max_tokens=1000, # 최대 토큰 수 지정
                            temperature=0.8, # 완성의 다양성을 조절하는 온도 설정
                            n=1 # 생성할 완성의 개수 지정
                            )

    assistant_reply = response.choices[0].message['content'] # 첫 번째 응답 결과 가져오기
    
    return assistant_reply # 응답 반환

[11장: 299페이지]

In [ ]:
k_text = "가구가 있는 거실" # 그림 생성 요청을 위한 텍스트
e_text = translate_text_for_image(k_text)
e_text

[11장: 300페이지]

In [ ]:
import textwrap

# OpenAI Image API((DALL·E)를 이용해 영어 문장으로 이미지를 생성하는 함수
def generate_image_from_text(text_for_image, image_num=1, image_size="512x512"):    
    # API 키 설정
    openai.api_key = os.environ["OPENAI_API_KEY"]
    
    shorten_text_for_image = textwrap.shorten(text_for_image, 1000) # 1,000자로 제한
    
    openai.api_key = os.environ["OPENAI_API_KEY"]
    response = openai.Image.create(prompt=shorten_text_for_image, n=image_num, size=image_size)
    
    image_urls = [] # 이미지 URL 리스트
    for data in response['data']:
        image_url = data['url'] # 이미지 URL 추출    
        image_urls.append(image_url)   
        
    return image_urls # 이미지 URL 리스트 반환

In [ ]:
import openai
import os
import requests
from IPython.display import Image, display

image_urls = generate_image_from_text(e_text) # 이미지 생성

download_holder = "C:/myPyAI/data/download/" # 다운로드 폴더 지정

for image_url in image_urls:
    image_file = image_url.split("?")[0].split("/")[-1]  # 이미지 파일 이름 추출   
    filename = download_holder + image_file # 다운로드 파일의 경로 생성
    print(filename)
    display(Image(image_url, format='png')) # 생성한 이미지의 링크를 이용해 이미지 표시
    
    # 생성한 이미지 다운로드
    r = requests.get(image_url) # 생성 이미지 URL 접속
    with open(filename, 'wb') as f: 
        f.write(r.content)    

## 11.2 상세 묘사로 이미지 생성

[11장: 303페이지]

In [ ]:
# OpenAI Chat Completions API를 이용해 이미지를 위한 상세 묘사를 생성하는 함수
def generate_text_for_image(text):
    # API 키 설정
    openai.api_key = os.environ["OPENAI_API_KEY"]

    # 대화 메시지 정의
    user_content = f"Describe the following in 1000 characters to create an image.\n {text}"
    
    messages = [ {"role": "user", "content": user_content} ]

    # Chat Completions API 호출
    response = openai.ChatCompletion.create(
                            model="gpt-3.5-turbo", # 사용할 모델 선택 
                            messages=messages, # 전달할 메시지 지정
                            max_tokens=1000, # 최대 토큰 수 지정
                            temperature=0.8, # 완성의 다양성을 조절하는 온도 설정
                            n=1 # 생성할 완성의 개수 지정
                        )

    assistant_reply = response.choices[0].message['content'] # 첫 번째 응답 결과 가져오기

    return assistant_reply # 응답 반환

[11장: 304페이지]

In [ ]:
from IPython.display import Image, display
import textwrap

k_text = "따뜻한 분위기가 나는 유럽풍의 거실 인테리어"
e_text = translate_text_for_image(k_text) # 한국어를 영어로 번역
print("[번역 텍스트]\n", e_text) # 응답받은 내용 출력

text_for_image = generate_text_for_image(e_text) # OpenAI ChatGPT에 질문하고 응답받음
shorten_text_for_image = textwrap.shorten(text_for_image, 250, placeholder=' [..이하 생략..]')
print("[이미지 생성을 위한 텍스트(축약)]\n", shorten_text_for_image) # 응답받은 내용 출력(축약)
# print("[이미지 생성을 위한 텍스트]\n", text_for_image) # 응답받은 내용 출력

image_num = 1 # 생성할 이미지 개수
image_urls = generate_image_from_text(text_for_image, image_num) # 지정한 개수만큼 이미지 생성
for image_url in image_urls:       
    display(Image(image_url, format='png')) # 이미지를 화면에 표시

[11장: 305페이지]

In [ ]:
%%writefile C:\myPyAI\code\my_image_gen.py
# 이미지 생성을 위한 모듈

import openai
import os
import textwrap

# OpenAI Chat Completions API를 이용해 한국어를 영어로 번역하는 함수
def translate_text_for_image(text):    
    # API 키 설정
    openai.api_key = os.environ["OPENAI_API_KEY"]

    # 대화 메시지 정의
    user_content = f"Translate the following Korean sentences into English.\n {text}"
    messages = [ {"role": "user", "content": user_content} ]

    # Chat Completions API 호출
    response = openai.ChatCompletion.create(
                            model="gpt-3.5-turbo", # 사용할 모델 선택 
                            messages=messages, # 전달할 메시지 지정
                            max_tokens=1000, # 최대 토큰 수 지정
                            temperature=0.8, # 완성의 다양성을 조절하는 온도 설정
                            n=1 # 생성할 완성의 개수 지정
                            )

    assistant_reply = response.choices[0].message['content'] # 첫 번째 응답 결과 가져오기
    
    return assistant_reply # 응답 반환

# OpenAI Chat Completions API를 이용해 이미지를 위한 상세 묘사를 생성하는 함수
def generate_text_for_image(text):
    # API 키 설정
    openai.api_key = os.environ["OPENAI_API_KEY"]

    # 대화 메시지 정의
    user_content = f"Describe the following in 1000 characters to create an image.\n {text}"
    
    messages = [ {"role": "user", "content": user_content} ]

    # Chat Completions API 호출
    response = openai.ChatCompletion.create(
                            model="gpt-3.5-turbo", # 사용할 모델 선택 
                            messages=messages, # 전달할 메시지 지정
                            max_tokens=1000, # 최대 토큰 수 지정
                            temperature=0.8, # 완성의 다양성을 조절하는 온도 설정
                            n=1 # 생성할 완성의 개수 지정
                        )
    
    assistant_reply = response.choices[0].message['content'] # 첫 번째 응답 결과 가져오기

    return assistant_reply # 응답 반환

# OpenAI Image API((DALL·E)를 이용해 영어 문장으로 이미지를 생성하는 함수
def generate_image_from_text(text_for_image, image_num=1, image_size="512x512"):    
    # API 키 설정
    
    shorten_text_for_image = textwrap.shorten(text_for_image, 1000) # 1,000자로 제한
    
    openai.api_key = os.environ["OPENAI_API_KEY"]
    response = openai.Image.create(prompt=shorten_text_for_image, n=image_num, size=image_size)
    
    image_urls = [] # 이미지 URL 리스트
    for data in response['data']:
        image_url = data['url'] # 이미지 URL 추출    
        image_urls.append(image_url)   
        
    return image_urls # 이미지 URL 리스트 반환


[11장: 307페이지]

In [ ]:
!echo %PYTHONPATH%

In [ ]:
import my_image_gen # 이미지 생성을 위한 모듈을 임포트
from IPython.display import Image, display
import textwrap

k_text = "새벽 바다 위에 배가 있고 해가 떠오르는 일출 풍경" # 한국어 입력
e_text = my_image_gen.translate_text_for_image(k_text) # 한국어를 영어로 번역
print("[번역 텍스트]\n", e_text) # 응답받은 내용 출력

text_for_image = my_image_gen.generate_text_for_image(e_text) # OpenAI ChatGPT에 질문하고 응답받음
shorten_text_for_image = textwrap.shorten(text_for_image, 250, placeholder=' [..이하 생략..]')
print("[이미지 생성을 위한 텍스트(축약)]\n", shorten_text_for_image) # 응답받은 내용 출력(축약)
# print("[이미지 생성을 위한 텍스트]\n", text_for_image) # 응답받은 내용 출력

image_num = 1 # 생성할 이미지 개수
image_size = "512x512" # 생성할 이미지 크기: "256x256", 512x512", "1024x1024"
image_urls = my_image_gen.generate_image_from_text(text_for_image, image_num, image_size) # 이미지 생성
for image_url in image_urls:       
    display(Image(image_url, format='png')) # 이미지를 화면에 표시

## 11.3 이미지를 생성하는 웹 앱

[11장: 309페이지]

In [ ]:
%%writefile C:\myPyAI\code\st\openai_image_app.py
# OpenAI 이미지 생성기 웹 앱

import my_image_gen # 이미지 생성을 위한 모듈을 임포트
import streamlit as st
import openai
import os
import requests
import textwrap
from datetime import datetime

# ---- 세션 상태 초기화 --------
if 'image_caption' not in st.session_state:
    st.session_state['image_caption'] = "" # 빈 문자열로 초기화 
    
if 'shorten_text_for_image' not in st.session_state:
    st.session_state['shorten_text_for_image'] = "" # 빈 문자열로 초기화 
    
if 'image_urls' not in st.session_state:
    st.session_state['image_urls'] = [] # 빈 리스트로 초기화
    
if 'images' not in st.session_state:
    st.session_state['images'] = [] # 빈 리스트로 초기화    

if 'download_file_names' not in st.session_state:
    st.session_state['download_file_names'] = [] # 빈 리스트로 초기화    
    
if 'download_buttons' not in st.session_state:
    st.session_state['download_buttons'] = False # False로 초기화
    
# ---- 이미지 생성을 위한 텍스트와 생성된 이미지를 화면에 표시하는 함수 ----
def display_results():
    # 저장한 세션 상태 불러오기
    shorten_text_for_image = st.session_state['shorten_text_for_image']
    image_caption = st.session_state['image_caption']
    image_urls = st.session_state['image_urls']    
    
    # 사이드바에 텍스트를 표시
    st.sidebar.write("[이미지 생성을 위한 텍스트]") 
    st.sidebar.write(shorten_text_for_image)
    
    # 이미지와 다운로드 버튼을 화면에 표시
    for k, image_url in enumerate(image_urls):
        st.image(image_url, caption=image_caption) # 이미지 표시
        
        image_data = st.session_state['images'][k]
        download_file_name = st.session_state['download_file_names'][k]

        # 다운로드 버튼
        st.download_button( label="이미지 파일 다운로드",
                            data=image_data,
                            file_name=download_file_name,
                            mime="image/png",
                            key=k,
                            on_click=download_button_callback)
        
# ------------------- 콜백 함수 --------------------
def download_button_callback():
    st.session_state['download_buttons'] = True

def button_callback():
    
    if radio_selected_lang == "한국어":
        translated_text = my_image_gen.translate_text_for_image(input_text) # 한국어를 영어로 번역
    elif radio_selected_lang == "영어":
        translated_text = input_text
    
    if detail_description == 'Yes':        
        resp = my_image_gen.generate_text_for_image(translated_text) # 이미지 생성을 위한 상세 묘사 생성
        text_for_image = resp
        image_caption ="상세 묘사를 추가해 생성한 이미지"
    elif detail_description == 'No': 
        text_for_image = translated_text
        image_caption ="입력 내용으로 생성한 이미지"
    
    # 텍스트 축약
    shorten_text_for_image = textwrap.shorten(text_for_image, 200, placeholder=' [..이하 생략..]')
    
    # 이미지 생성
    image_urls = my_image_gen.generate_image_from_text(text_for_image, image_num, image_size)

    # 이미지와 다운로드 파일 이름 생성
    images = []
    download_file_names = []
    for k, image_url in enumerate(image_urls):
        
        # 생성한 이미지 다운로드
        r = requests.get(image_url)
        image_data = r.content # 이미지 데이터
        images.append(image_data)
        
        # 다운로드 파일 이름 생성
        now_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") # 이미지 생성 날짜와 시각
        download_file_name = f"gen_image_{k}_{now_datetime}.png"
        download_file_names.append(download_file_name)
        
    # 세션 상태 저장
    st.session_state['image_caption'] = image_caption
    st.session_state['shorten_text_for_image'] = shorten_text_for_image
    st.session_state['image_urls'] = image_urls
    st.session_state['download_file_names'] = download_file_names
    st.session_state['images'] = images

# ------------- 사이드바 화면 구성 --------------------------   
st.sidebar.title("이미지 생성을 위한 설정 ")

input_text = st.sidebar.text_input("이미지 생성을 위한 설명을 입력하세요.",
                                    "빌딩이 보이는 호수가 있는 도시의 공원")

radio_selected_lang = st.sidebar.radio('입력한 언어', ['한국어', '영어'], 
                                       index=0, horizontal=True)

# 라디오 버튼: 생성 이미지 개수 지정
image_num_options = [1, 2, 3] # 세 종류의 이미지 개수 선택 가능
image_num = st.sidebar.radio('생성할 이미지 개수를 선택하세요.', 
                      image_num_options, index=0, horizontal=True)

# 라디오 버튼: 이미지 크기 지정
image_size_options = ['256x256', '512x512', '1024x1024'] # 세 종류의 이미지 크기 선택 가능
image_size = st.sidebar.radio('생성할 이미지 크기를 선택하세요.', 
                      image_size_options, index=1, horizontal=True)

# 라디오 버튼: 상세 묘사 추가 여부 지정
detail_description = st.sidebar.radio('상세 묘사를 추가하겠습니까?', 
                      ['Yes', 'No'], index=1, horizontal=True)

# 기본 버튼: 이미지 생성을 위해 사용
# clicked = st.sidebar.button('이미지 생성')
clicked = st.sidebar.button('이미지 생성', on_click=button_callback)

# ------------- 메인 화면 구성 --------------------------   
st.title("인공지능 이미지 생성기")

# [이미지 생성] 버튼 혹은 [이미지 파일 다운로드] 버튼 클릭 시 화면 표시 함수 실행    
if clicked or st.session_state['download_buttons'] == True:
    display_results()

## 11.4 정리